In [1]:
import numpy as np
import pandas as pd
from src.algorithms import * 
from src.utils import * 
from src.metrics_FAMD import *

## TEST on a generated Dataset

In [5]:
df = pd.read_csv("df_3_1.csv") # first dataset created in the paper : (3.1 Relationships between continuous and categorical variables)

n_it =1000
print(df.shape)

(100, 6)


In [6]:
# Categorical Variables :
idx_k2_df = pd.Index(["1","2"])

# Continuous Variables
idx_k1_df = df.columns.difference(idx_k2_df)


In [7]:
# Inject missing values into the gbsg dataframe :
proba_non_missing = 0.8
df_missing = create_missingness(df, proba_non_missing)

# Encode dummy variables in gbsg dataframe and in the gbsg dataframe with missing values :
df_missing_dummy, idx_j_df, nb_values_per_cat_df = encode_dummy_variables(df_missing, idx_k2_df)
df_dummy = encode_dummy_variables(df, idx_k2_df)[0]

In [8]:
ifamd_df = IterativeFAMDImputer(n_components=4, data=df_missing_dummy, k1=idx_k1_df, k2=idx_j_df, nb_values_per_cat = nb_values_per_cat_df)
ifamd_df.impute(n_it)

Converged in 9


,0,3,4,5,1_0.0,1_1.0,1_3.0,1_2.0,2_0.0,2_2.0,2_1.0,2_3.0
0,0.093337,0.135113,0.167152,0.197447,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
1,-1.072932,1.061793,1.051826,1.134287,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
2,0.177369,0.414308,0.415502,0.442817,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
3,-0.608092,0.327232,0.439984,0.327426,0.403212,-0.099130,0.941222,-0.245304,0.000000,1.000000,0.000000,0.000000
4,1.775482,-0.293381,-0.248346,-0.257979,0.000000,0.000000,1.000000,0.000000,-0.514017,1.384534,-0.163783,0.293266
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.289194,-0.401571,-0.320624,-0.361769,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
96,1.227742,0.083648,-0.155891,-0.145247,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
97,-1.009023,-0.190538,-0.186147,-0.259537,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
98,-1.664287,-1.140274,-1.182114,-1.254136,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [9]:
C0_missing = df_missing_dummy.isna()[ifamd_df.k1].to_numpy()
Categ_missing = df_missing_dummy.isna()[ifamd_df.k2].to_numpy()

# We encode categories into 0,1
res = ifamd_df.df[ifamd_df.k2].copy()
pos = 0
for h in range (len(idx_k2_df)) :
    col = [idx_j_df[pos+i] for i in range (nb_values_per_cat_df[h])]
    res["max_value"] = ifamd_df.df[col].max(axis = 1)
    for value in col:
        res[value] = (res[value] == res["max_value"]).astype(int)

    pos += nb_values_per_cat_df[h]

res = res[ifamd_df.k2] 

metric_fc(res[Categ_missing], df_dummy[ifamd_df.k2][Categ_missing])

array([0.18518519, 0.11111111, 0.22222222, 0.14814815, 0.33333333,
       0.18518519, 0.22222222, 0.22222222])

In [10]:
#NRMSE
compute_nrmse_weighted(df_dummy[ifamd_df.k1][C0_missing], ifamd_df.df[ifamd_df.k1][C0_missing])

6.323510807689512